In [ ]:
import gymnasium as gym
import torch as th
from LunarLander3DEnv.envs import LunarLander3DEnv  # Pastikan environment sudah terdaftar (register)

# Import algoritma dari stable-baselines3
from stable_baselines3 import PPO, A2C, DDPG, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

# Import tambahan untuk TRPO, RecurrentPPO, dan HER
from sb3_contrib import TRPO
# Untuk RecurrentPPO, import dilakukan hanya jika diperlukan

# Pilih algoritma yang ingin digunakan:
# Pilihan: "PPO", "A2C", "DDPG", "SAC", "TD3", "TRPO", "RecurrentPPO", atau "HER"
algorithm = "RecurrentPPO"  # Ganti sesuai kebutuhan, misal "RecurrentPPO" atau "HER"

# Buat environment
env = gym.make("LunarLander3DEnv-v0", render_mode=None)

# Inisialisasi model sesuai algoritma yang dipilih
model = None

if algorithm == "PPO":
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.01,
        vf_coef=0.5,
        policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256])),
        verbose=1,
        device="cuda" if th.cuda.is_available() else "cpu"
    )

elif algorithm == "SAC":
    model = SAC(
        "MlpPolicy",
        env,
        learning_rate=3e-4,
        buffer_size=500_000,
        batch_size=128,
        tau=0.005,
        gamma=0.99,
        ent_coef="auto_0.1",
        use_sde=True,
        policy_kwargs=dict(net_arch=dict(pi=[256, 256], qf=[256, 256])),
        verbose=1,
    )

elif algorithm == "TD3":
    model = TD3(
        "MlpPolicy",
        env,
        learning_rate=1e-3,
        buffer_size=500_000,
        batch_size=128,
        tau=0.005,
        gamma=0.99,
        policy_delay=2,
        target_policy_noise=0.2,
        target_noise_clip=0.5,
        policy_kwargs=dict(net_arch=[256, 256]),
        verbose=1,
    )

elif algorithm == "TRPO":
    model = TRPO(
        "MlpPolicy",
        env,
        learning_rate=1e-4,
        gamma=0.99,
        gae_lambda=0.95,
        #max_kl=0.01,
        #ent_coef=0.01,
        #vf_coef=0.5,
        policy_kwargs=dict(net_arch=[256, 256]),
        verbose=1,
    )

elif algorithm == "RecurrentPPO":
    from sb3_contrib import RecurrentPPO  # Pastikan sudah install sb3-contrib versi terbaru
    model = RecurrentPPO(
        "MlpLstmPolicy",  # Menggunakan policy dengan LSTM
        env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.01,
        vf_coef=0.5,
        policy_kwargs=dict(lstm_hidden_size=256, net_arch=[256, 256]),
        verbose=1,
        device="cuda" if th.cuda.is_available() else "cpu"
    )

elif algorithm == "HER":
    # HER biasanya digunakan pada environment goal-oriented. Pastikan env sudah sesuai
    from stable_baselines3 import HER, SAC  # Contoh: gunakan SAC sebagai algoritma dasar untuk HER
    model = HER(
        "MlpPolicy",
        env,
        SAC,
        n_sampled_goal=4,
        goal_selection_strategy="future",  # Bisa juga 'final' atau 'episode'
        online_sampling=True,
        verbose=1,
        device="cuda" if th.cuda.is_available() else "cpu"
    )

else:
    raise ValueError("Algoritma yang dipilih tidak didukung. Pilih salah satu: PPO, SAC, TD3, TRPO, RecurrentPPO, atau HER.")

# Setup callbacks untuk menyimpan model dan evaluasi secara berkala
total_timesteps_learning = 100_000

checkpoint_callback = CheckpointCallback(
    save_freq=int(total_timesteps_learning / 10),
    save_path='./models/',
    name_prefix=f'{algorithm}_checkpoint_v1_'
)
eval_callback = EvalCallback(
    env,
    best_model_save_path="./logs/",
    log_path="./logs/",
    eval_freq=total_timesteps_learning // 10,
    n_eval_episodes=10,
    deterministic=True,
)

# Mulai proses pelatihan
model.learn(total_timesteps=total_timesteps_learning, callback=[checkpoint_callback, eval_callback])

# Simpan model yang sudah dilatih
model.save(f"ll3d_{algorithm}")
print("Training is finished")

# Evaluasi agent
n_eval_episodes = 10
render = False  # Ubah ke True jika ingin melihat visualisasi evaluasi
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=n_eval_episodes, render=render)

print(f"Mean Reward over {n_eval_episodes} episodes: {mean_reward:.2f} +/- {std_reward:.2f}")

env.close()
del model


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 378       |
|    ep_rew_mean     | -1.06e+05 |
| time/              |           |
|    fps             | 172       |
|    iterations      | 1         |
|    time_elapsed    | 11        |
|    total_timesteps | 2048      |
----------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 318           |
|    ep_rew_mean          | -8.8e+04      |
| time/                   |               |
|    fps                  | 77            |
|    iterations           | 2             |
|    time_elapsed         | 52            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00035241139 |
|    clip_fraction        | 0             |
|    clip_range           |

c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=-144567.72 +/- 69496.74
Episode length: 474.00 +/- 214.61
-------------------------------------------
| eval/                   |               |
|    mean_ep_length       | 474           |
|    mean_reward          | -1.45e+05     |
| time/                   |               |
|    total_timesteps      | 10000         |
| train/                  |               |
|    approx_kl            | 0.00018357154 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -29.9         |
|    explained_variance   | 5.13e-06      |
|    learning_rate        | 0.0003        |
|    loss                 | 1.38e+07      |
|    n_updates            | 40            |
|    policy_gradient_loss | -0.000971     |
|    std                  | 1             |
|    value_loss           | 2.77e+07      |
-------------------------------------------
New best mean reward!
----------------------------------
| rollou

In [ ]:
import gymnasium as gym
import torch as th
from LunarLander3DEnv.envs import LunarLander3DEnv
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.callbacks import BaseCallback

class CurriculumCallback(BaseCallback):
    def __init__(self, total_timesteps, verbose=0):
        super(CurriculumCallback, self).__init__(verbose)
        self.total_timesteps = total_timesteps

    def _on_step(self) -> bool:
        # Hitung progres (0 hingga 1)
        progress = self.num_timesteps / self.total_timesteps
        # Misal, kita naikkan difficulty secara linier dari 0 ke 1
        for env in self.model.get_env().envs:
            if hasattr(env, "difficulty"):
                env.difficulty = progress
        return True

# Fungsi untuk membuat environment
def make_env(render_mode=None):
    def _init():
        env = gym.make("LunarLander3DEnv-v0", render_mode=render_mode)
        # Inisialisasi difficulty ke 0
        env.difficulty = 0.0
        return env
    return _init

num_envs = 8
train_envs = DummyVecEnv([make_env(render_mode=None) for _ in range(num_envs)])
eval_env = DummyVecEnv([make_env(render_mode=None)])

total_timesteps_learning = 1_000_000

# Inisialisasi model PPO (misalnya)
model = RecurrentPPO(
    "MlpLstmPolicy",
    train_envs,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    vf_coef=0.5,    
    policy_kwargs=dict(
        net_arch=dict(pi=[256, 256], vf=[256, 256]),
        lstm_hidden_size=256,
        n_lstm_layers=2,
        shared_lstm=True,
        enable_critic_lstm=False,  # Pastikan LSTM kritikus tidak diaktifkan
    ),
    verbose=1,
    device="cuda" if th.cuda.is_available() else "cpu"
)

checkpoint_callback = CheckpointCallback(
    save_freq=int(total_timesteps_learning / 10),
    save_path='./models/',
    name_prefix='rppo_curriculum_checkpoint_'
)

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./logs/",
    log_path="./logs/",
    eval_freq=int(total_timesteps_learning / 10),
    n_eval_episodes=10,
    deterministic=True,
)

curriculum_callback = CurriculumCallback(total_timesteps=total_timesteps_learning)

# Training dengan callback gabungan
model.learn(
    total_timesteps=total_timesteps_learning,
    callback=[checkpoint_callback, eval_callback, curriculum_callback],
    progress_bar=True
)

model.save("ll3d_rppo_curriculum")
train_envs.close()
eval_env.close()


Using cuda device


Output()

------------------------------
| time/              |       |
|    fps             | 58    |
|    iterations      | 1     |
|    time_elapsed    | 281   |
|    total_timesteps | 16384 |
------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 54          |
|    iterations           | 2           |
|    time_elapsed         | 603         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.010321844 |
|    clip_fraction        | 0.0705      |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.9       |
|    explained_variance   | 3.1e-06     |
|    learning_rate        | 0.0003      |
|    loss                 | 57.1        |
|    n_updates            | 10          |
|    policy_gradient_loss | 0.00135     |
|    std                  | 1.01        |
|    value_loss           | 9.18e+03    |
-----------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 53          |
|    iterations           | 3           |
|    time_elapsed         | 926         |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.004619338 |
|    clip_fraction        | 0.0496      |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.9       |
|    explained_variance   | 5.19e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 187         |
|    n_updates            | 20          |
|    policy_gradient_loss | 0.00224     |
|    std                  | 1.01        |
|    value_loss           | 1.38e+04    |
-----------------------------------------


...

-----------------------------------------
| time/                   |             |
|    fps                  | 52          |
|    iterations           | 45          |
|    time_elapsed         | 13976       |
|    total_timesteps      | 737280      |
| train/                  |             |
|    approx_kl            | 0.010400822 |
|    clip_fraction        | 0.116       |
|    clip_range           | 0.2         |
|    entropy_loss         | -34         |
|    explained_variance   | 0.478       |
|    learning_rate        | 0.0003      |
|    loss                 | 58.4        |
|    n_updates            | 440         |
|    policy_gradient_loss | -0.000454   |
|    std                  | 1.22        |
|    value_loss           | 106         |
-----------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 52          |
|    iterations           | 46          |
|    time_elapsed         | 14265       |
|    total_timesteps      | 753664      |
| train/                  |             |
|    approx_kl            | 0.012366281 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -34.1       |
|    explained_variance   | 0.685       |
|    learning_rate        | 0.0003      |
|    loss                 | 107         |
|    n_updates            | 450         |
|    policy_gradient_loss | -0.000826   |
|    std                  | 1.23        |
|    value_loss           | 130         |
-----------------------------------------


----------------------------------------
| time/                   |            |
|    fps                  | 52         |
|    iterations           | 47         |
|    time_elapsed         | 14554      |
|    total_timesteps      | 770048     |
| train/                  |            |
|    approx_kl            | 0.00970849 |
|    clip_fraction        | 0.114      |
|    clip_range           | 0.2        |
|    entropy_loss         | -34.2      |
|    explained_variance   | 0.799      |
|    learning_rate        | 0.0003     |
|    loss                 | 3.37       |
|    n_updates            | 460        |
|    policy_gradient_loss | -0.00152   |
|    std                  | 1.24       |
|    value_loss           | 90.2       |
----------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 52          |
|    iterations           | 48          |
|    time_elapsed         | 14843       |
|    total_timesteps      | 786432      |
| train/                  |             |
|    approx_kl            | 0.009372419 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -34.3       |
|    explained_variance   | 0.536       |
|    learning_rate        | 0.0003      |
|    loss                 | 42.8        |
|    n_updates            | 470         |
|    policy_gradient_loss | -0.00243    |
|    std                  | 1.24        |
|    value_loss           | 139         |
-----------------------------------------


c:\Users\User\anaconda3\envs\robot_sim\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: 
Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode 
lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor``
wrapper.
  warnings.warn(

In [ ]:
import gymnasium as gym
import torch as th
from LunarLander3DEnv.envs import LunarLander3DEnv
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

num_envs = 8  # Jumlah environment paralel

def make_env(render_mode=None):
    def _init():
        return gym.make("LunarLander3DEnv-v0", render_mode=render_mode)
    return _init

train_envs = DummyVecEnv([make_env(render_mode=None) for _ in range(num_envs)])
eval_env = DummyVecEnv([make_env(render_mode=None)])

model = RecurrentPPO(
    "MlpLstmPolicy",
    train_envs,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    vf_coef=0.5,    
    policy_kwargs=dict(
        net_arch=dict(pi=[256, 256], vf=[256, 256]),
        lstm_hidden_size=256,
        n_lstm_layers=2,
        shared_lstm=True,
        enable_critic_lstm=False,  # Pastikan LSTM kritikus tidak diaktifkan
    ),
    verbose=1,
    device="cuda" if th.cuda.is_available() else "cpu"
)

total_timesteps_learning = 500_000

checkpoint_callback = CheckpointCallback(
    save_freq=int(total_timesteps_learning / 10),
    save_path='./models/',
    name_prefix='RecurrentPPO_parallel_checkpoint_'
)

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./logs/",
    log_path="./logs/",
    eval_freq=int(total_timesteps_learning / 10),
    n_eval_episodes=10,
    deterministic=True,
)

model.learn(
    total_timesteps=total_timesteps_learning,
    callback=[checkpoint_callback, eval_callback],
    progress_bar=True
)

model.save("ll3d_RecurrentPPO_parallel")
train_envs.close()
eval_env.close()


Using cuda device


Output()

------------------------------
| time/              |       |
|    fps             | 53    |
|    iterations      | 1     |
|    time_elapsed    | 304   |
|    total_timesteps | 16384 |
------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 49          |
|    iterations           | 2           |
|    time_elapsed         | 660         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.007973919 |
|    clip_fraction        | 0.0737      |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.9       |
|    explained_variance   | 2.8e-05     |
|    learning_rate        | 0.0003      |
|    loss                 | 1.37e+05    |
|    n_updates            | 10          |
|    policy_gradient_loss | 0.00118     |
|    std                  | 1.01        |
|    value_loss           | 9.13e+04    |
-----------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 49          |
|    iterations           | 3           |
|    time_elapsed         | 992         |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.039660174 |
|    clip_fraction        | 0.135       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.9       |
|    explained_variance   | 1.38e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.61e+04    |
|    n_updates            | 20          |
|    policy_gradient_loss | 0.00674     |
|    std                  | 1.01        |
|    value_loss           | 9.28e+04    |
-----------------------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 50           |
|    iterations           | 4            |
|    time_elapsed         | 1305         |
|    total_timesteps      | 65536        |
| train/                  |              |
|    approx_kl            | 0.0072901165 |
|    clip_fraction        | 0.108        |
|    clip_range           | 0.2          |
|    entropy_loss         | -30          |
|    explained_variance   | 9.24e-06     |
|    learning_rate        | 0.0003       |
|    loss                 | 2.56e+04     |
|    n_updates            | 30           |
|    policy_gradient_loss | 0.00625      |
|    std                  | 1.01         |
|    value_loss           | 6.23e+04     |
------------------------------------------


In [2]:
import gymnasium as gym
import torch as th
from LunarLander3DEnv.envs import LunarLander3DEnv
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
import time
algorithm = "RPPO"
model_path = f"ll3d_rppo_curriculum"  # Nama file model yang telah disimpan

# Buat environment dengan render_mode "human" agar dapat melihat visualisasi
env = gym.make("LunarLander3DEnv-v0", render_mode="human")
model = RecurrentPPO.load(model_path, env=env)

# Jalankan beberapa episode untuk inferensi
num_episodes = 10
for episode in range(1, num_episodes+1):
    obs, info = env.reset()
    done = False
    total_reward = 0.0
    while not done:
        # Prediksi aksi dengan deterministik
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        env.render()  # Pastikan render dipanggil agar kamera dan visualisasi diupdate
        #time.sleep(1/60)  # Optional: delay untuk melambatkan tampilan
        done = terminated or truncated
        time.sleep(0.01)
    print(f"Episode {episode}: Total Reward = {total_reward:.2f}")

env.close()


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Episode 1: Total Reward = -47063.38


error: Not connected to physics server.